# Question1 
Re-analyse the annual maximum gust wind speeds for (I) the years 1952 to 1998, (II) the years 1952 to 1997, i.e. ignore the high value recorded in 1998. Compare the resulting predictions of design wind speeds for (a) 50 years return period, and (b) 1000 years return period, and comment.
# Question2 
Using the parameter estimation approach in code GB 50009 2012, predict the 10, 20, 50, 100, 200, 500, 1000 years return period design wind speeds for the above two cases, and compare the results with the Gumbel approach.


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
from io import StringIO

# 加载数据
data = """
year,Wind speed m/s
1952,31.4
1953,33.4
1954,29.8
1955,30.3
1956,27.8
1957,30.3
1958,29.3
1959,36.5
1960,29.3
1961,27.3
1962,31.9
1963,28.8
1964,25.2
1965,27.3
1966,23.7
1967,27.8
1968,32.4
1969,27.8
1970,26.2
1971,30.9
1972,31.9
1973,27.3
1974,25.7
1975,32.9
1976,28.3
1977,27.3
1978,28.3
1979,28.3
1980,29.3
1981,27.8
1982,27.8
1983,30.9
1984,26.7
1985,30.3
1986,28.3
1987,30.3
1988,34
1989,28.8
1990,30.3
1991,27.3
1992,27.8
1993,28.8
1994,30.9
1995,26.2
1996,25.7
1997,24.7
1998,42.2
"""

# Answer for Question1

In [2]:
df = pd.read_csv(StringIO(data)).sort_values('Wind speed m/s').reset_index(drop=True)
df_1952_1997 = df[df.year < 1998]

# 缩减变量计算函数
def calculate_reduced_variate(data, method='gumbel'):
    N = len(data)
    m = np.arange(1, N+1)
    if method == 'gumbel':
        p = m/(N+1)
    elif method == 'gringorten':
        p = (m-0.44)/(N+0.12)
    return -np.log(-np.log(p))

# 设计风速计算函数
def design_wind_speed(return_period, slope, intercept):
    y = -np.log(-np.log(1 - 1/return_period))
    return intercept + slope * y

# 计算回归参数
def calculate_regression(data, method):
    x = calculate_reduced_variate(data['Wind speed m/s'], method)
    slope, intercept = linregress(x, data['Wind speed m/s'])[:2]
    return slope, intercept, x

def solve_problem_1():
    # Gumbel方法
    gumbel_slope_1998, gumbel_intercept_1998, _ = calculate_regression(df, 'gumbel')
    gumbel_slope_1997, gumbel_intercept_1997, _ = calculate_regression(df_1952_1997, 'gumbel')
    
    # Gringorten方法
    gringorten_slope_1998, gringorten_intercept_1998, _ = calculate_regression(df, 'gringorten')
    gringorten_slope_1997, gringorten_intercept_1997, _ = calculate_regression(df_1952_1997, 'gringorten')
    
    # 计算设计风速
    return_periods = [50, 1000]
    results = {}
    for period in return_periods:
        results[period] = {
            'gumbel_1998': design_wind_speed(period, gumbel_slope_1998, gumbel_intercept_1998),
            'gumbel_1997': design_wind_speed(period, gumbel_slope_1997, gumbel_intercept_1997),
            'gringorten_1998': design_wind_speed(period, gringorten_slope_1998, gringorten_intercept_1998),
            'gringorten_1997': design_wind_speed(period, gringorten_slope_1997, gringorten_intercept_1997)
        }
    
    # 打印结果
    print("="*50 + "\n问题1：设计风速比较\n" + "="*50)
    for period in return_periods:
        print(f"\n{period}年回归周期:")
        print(f"Gumbel (1952-1998): {results[period]['gumbel_1998']:.2f} m/s")
        print(f"Gumbel (1952-1997): {results[period]['gumbel_1997']:.2f} m/s")
        print(f"Gringorten (1952-1998): {results[period]['gringorten_1998']:.2f} m/s")
        print(f"Gringorten (1952-1997): {results[period]['gringorten_1997']:.2f} m/s")

# 调用问题1的解决函数
solve_problem_1()

问题1：设计风速比较

50年回归周期:
Gumbel (1952-1998): 38.19 m/s
Gumbel (1952-1997): 36.34 m/s
Gringorten (1952-1998): 37.64 m/s
Gringorten (1952-1997): 35.82 m/s

1000年回归周期:
Gumbel (1952-1998): 46.18 m/s
Gumbel (1952-1997): 42.93 m/s
Gringorten (1952-1998): 45.20 m/s
Gringorten (1952-1997): 41.98 m/s


# Answer for Question2

In [3]:
# 解决问题2：对比Gumbel和GB 50009-2012的计算结果
def solve_problem_2():
    return_periods = [10, 20, 50, 100, 200, 500, 1000]
    
    # ==================== Gumbel方法计算 ====================
    def gumbel_design_speeds(data, return_periods):
        slope, intercept, _ = calculate_regression(data, 'gumbel')
        return [design_wind_speed(period, slope, intercept) for period in return_periods]
    
    # 计算Gumbel方法结果
    gumbel_results_1952_1998 = gumbel_design_speeds(df, return_periods)
    gumbel_results_1952_1997 = gumbel_design_speeds(df_1952_1997, return_periods)
    
    # ==================== GB 50009-2012方法计算 ====================
    def gb_design_speeds(data, return_periods, N):
        mean = data['Wind speed m/s'].mean()
        std = data['Wind speed m/s'].std()
        coefficients = {47: (1.155374, 0.547192), 46: (1.153612, 0.546746)}
        
        if N not in coefficients:
            raise ValueError("N值无效，应为46或47")
        
        c1, c2 = coefficients[N]
        alpha = c1 / std
        u = mean - c2 / alpha
        
        return [u + (-np.log(-np.log(1 - 1/p))) / alpha for p in return_periods]
    
    # 计算GB方法结果
    gb_results_1952_1998 = gb_design_speeds(df, return_periods, N=47)
    gb_results_1952_1997 = gb_design_speeds(df_1952_1997, return_periods, N=46)
    
    # ==================== 结果对比与输出 ====================
    print("="*50 + "\n问题2：Gumbel与GB 50009-2012设计风速对比\n" + "="*50)
    
    # 创建结果表格
    results = pd.DataFrame({
        'Return Period': return_periods,
        'Gumbel (1952-1998)': gumbel_results_1952_1998,
        'GB (1952-1998)': gb_results_1952_1998,
        'Gumbel (1952-1997)': gumbel_results_1952_1997,
        'GB (1952-1997)': gb_results_1952_1997
    }).set_index('Return Period')
    
    # 打印表格
    print("\n设计风速对比表（单位：m/s）：")
    print(results.round(2))

# 调用问题2的解决函数
solve_problem_2()

问题2：Gumbel与GB 50009-2012设计风速对比

设计风速对比表（单位：m/s）：
               Gumbel (1952-1998)  GB (1952-1998)  Gumbel (1952-1997)  \
Return Period                                                           
10                          33.79           33.98               32.72   
20                          35.71           35.97               34.30   
50                          38.19           38.55               36.34   
100                         40.04           40.48               37.87   
200                         41.89           42.40               39.40   
500                         44.33           44.94               41.41   
1000                        46.18           46.86               42.93   

               GB (1952-1997)  
Return Period                  
10                      32.79  
20                      34.40  
50                      36.48  
100                     38.04  
200                     39.60  
500                     41.65  
1000                    43.20  
